In [ ]:
import scipy.signal
import torch
from src.utils.fourier import dft, idft
import matplotlib.pyplot as plt
import scipy
import numpy as np
# from src.utils.filters import MovingAvg
from src.utils.fourier import sphere2complex, complex2sphere

# def moving_average_freq_response(N, sample_rate, freq):
#     omega = 2 * torch.pi * freq / sample_rate
#     # SMA coefficients
#     b = np.ones(N)
#     a = np.array([N] + [0]*(N-1))

#     # Calculate the frequency response
#     w, h = scipy.signal.freqz(b, a, worN=omega)
#     # w *= sample_rate / (2 * np.pi)                      # Convert from rad/sample to Hz
#     return h

def moving_average_freq_response(N, sample_rate, freq):
    omega = 2 * torch.pi * freq / sample_rate
    coeff = torch.exp(-1j * omega * (N - 1) / 2) / N
    omega = torch.where(omega == 0, 1e-5, omega)
    Hw = coeff * torch.sin(omega * N / 2) / torch.sin(omega / 2)
    # Hw = Hw*torch.exp(1j*omega*N)
    return Hw

# def moving_avg(x, N):
#     avg = torch.nn.AvgPool1d(kernel_size=N, stride=1)
#     front = x[:, 0:1, :].repeat(1, N // 2, 1)
#     end = x[:, -1:, :].repeat(1, N - 1 - N // 2, 1)
#     x = torch.cat([front, x, end], dim=1)
#     x = avg(x.permute(0, 2, 1))
#     x = x.permute(0, 2, 1)
#     return x



original signal

In [ ]:
t = torch.linspace(0, 4*torch.pi, 200)
sample_rate = len(t)/(t.max()-t.min())
x = torch.sin(t) + torch.sin(2*t) + 0.3*torch.sin(12*t) - 10
x_freq = torch.fft.rfft(x)

theta, phi = complex2sphere(x_freq.real, x_freq.imag)

freq = torch.fft.rfftfreq(len(t), 1/sample_rate)
mag = x_freq.abs()
phase = x_freq.angle()
phase[torch.abs(phase) < 1] = 0
fig, axs = plt.subplots(2)
axs[0].plot(t, x)
axs[1].plot(freq, x_freq.abs())
axs[1].set_yscale('log')
# axs[1].set_xscale('log')
# axs[2].plot(freq, x_freq.angle())
# axs[1].plot(freq, theta)
# axs[2].plot(freq, phi)
fig.tight_layout()



manuplate on the frequency domain

In [ ]:
N = 50
h = moving_average_freq_response(N, sample_rate=sample_rate, freq=freq)
x_freq_new = x_freq * h
# x_freq_new = x_freq * h * torch.exp(torch.tensor(-1j*N))
mag = x_freq_new.abs()
phase = x_freq_new.angle()
x_new = torch.fft.irfft(x_freq_new)
x_new_avgpool = torch.nn.functional.avg_pool1d(x.reshape(1,1,-1), N,N).permute(0,2,1)
fig, axs = plt.subplots(2)
# axs[0].plot(x_new)
axs[0].plot(t, x_new)
# axs[0].scatter(t[N-1::N],x_new[N-1::N])
# axs[0].scatter(t[N-1::N], x_new_avgpool.flatten())
axs[1].plot(freq, mag)
axs[1].set_yscale('log')
# axs[1].set_xscale('log')
# axs[2].plot(freq, phase)

manuplate on the time domain

In [ ]:
from src.utils.filters import MovingAvgTime


x_new = MovingAvgTime(N)(x.reshape(1,-1,1))
# x_new = np.convolve(np.concatenate([x[-N+1:],x]), np.ones(N)/N, mode='valid')
x_freq = torch.fft.rfft(x_new.flatten())
freq = torch.fft.rfftfreq(len(t), 1/sample_rate)
mag = x_freq.abs()
phase = x_freq.angle()

# mag = x_freq_new.abs()
# phase = x_freq_new.angle()
# # print(phase)
# x = torch.fft.irfft(x_freq_new)
# print(x_new[N//2:-N//2])
fig, axs = plt.subplots(3)
axs[0].plot(x_new.flatten())
axs[1].plot(freq, mag)
axs[1].set_yscale('log')
# axs[1].set_xscale('log')
axs[2].plot(freq, phase)
fig.tight_layout()

In [ ]:
class FreqLinear(torch.nn.Module):
    def __init__(self, in_channels, fft_len, kernal_size) -> None:
        super().__init__()
        self.kernel = torch.nn.Parameter(
            torch.complex(
                torch.ones(fft_len, kernal_size), torch.zeros(fft_len, kernal_size)
            ),
            requires_grad=True,
        )
        self.linear = torch.nn.Linear(kernal_size * in_channels , in_channels).to(torch.cfloat)
        self.in_channels= in_channels
        self.fft_len = fft_len
        
    def forward(self, x):
        # x = x.permute(0,2,1)
        x = torch.fft.rfft(x, norm="ortho", dim=1)
        x_channels = []
        for i in range(x.shape[-1]):
            x_channels.append(x[...,[i]] * self.kernel)
        x = torch.concat(x_channels, dim=-1)
        x = self.linear(x)
        # x = self.linear(x.flatten()).reshape(-1, self.fft_len, self.in_channels)
        x = torch.fft.irfft(x, norm="ortho", dim=1)
        return x


class TimeConv(torch.nn.Module):
    def __init__(self, in_channel) -> None:
        super().__init__()
        self.conv = torch.nn.Conv1d(
            in_channel, out_channels=in_channel, kernel_size=3, bias=False, padding=1
        )

    def forward(self, x):
        # x = x.permute(0,2,1)
        x = self.conv(x)
        return x


x = torch.sin(torch.arange(288) / 100).reshape(1, -1, 2)
m = FreqLinear(in_channels=x.shape[-1], fft_len=x.shape[1]//2+1, kernal_size=10)
for p in m.parameters():
    print(p.numel())
y = torch.ones_like(x)
loss_fn = torch.nn.MSELoss()
loss = loss_fn(y, m(x))
loss.backward()

# x = x.repeat(2,1,1)
# kernel = torch.rand(x.shape[1] // 2 + 1, 3)
# x = torch.fft.rfft(x, dim=1)
# print(x)
# print(kernel)
# print(x.shape)
# print(kernel.shape)
# x * kernel

In [ ]:
import torch
x = torch.ones(100) * 10
# x = torch.randn(1000)
# x = torch.sin(torch.arange(314 * 4) / 100)
x_fft = torch.fft.rfft(x)
plt.plot(x_fft.abs())
plt.yscale("log")
x_fft = x_fft + torch.randn_like(x_fft,dtype=torch.cfloat)
# plt.xscale("log")
plt.plot(x_fft.abs())

fig = plt.figure()
x = torch.fft.irfft(x_fft)
plt.plot(x)

In [ ]:
import torch
def moving_average_1d(data: torch.Tensor, kernel_size:int, stride:int=1):
    # Unfold the data tensor to create a windowed view
    unfolded_data = data.unfold(1, kernel_size, stride)
    print(unfolded_data.shape)
    
    # Create a kernel tensor for matrix multiplication
    kernel = torch.ones(kernel_size) / kernel_size
    
    # Perform matrix multiplication
    moving_averages_unfolded = unfolded_data @ kernel
    
    # # Pad the moving averages tensor to match the original data size
    # padding = kernel_size - 1
    # moving_averages_padded = torch.cat((torch.zeros(padding), moving_averages_unfolded))
    
    # Fold the moving averages tensor to get the final result
    # moving_averages = moving_averages_padded.view(-1)
    
    return moving_averages_unfolded

# Example usage
data = torch.arange(60).float().reshape(2,-1,3)  # 1D tensor with 10 elements
kernel_size = 5

moving_averages = moving_average_1d(data, kernel_size)
print(moving_averages)  # Output: torch.Size([10])

In [ ]:
from src.schedules.collection import linear_schedule
import torch
import matplotlib.pyplot as plt
import pandas as pd
from src.utils.filters import MovingAvgTime, MovingAvgFreq
from src.utils.fourier import complex2sphere, sphere2complex
from scipy.stats import norm

df = pd.read_csv(
    "/home/user/data/FrequencyDiffusion/dataset/MFRED_clean.csv",
    index_col=0,
    parse_dates=True,
)[["value"]]
df = df[:288]
x = torch.from_numpy(df.values).reshape(1, -1, 1)


# x_norm_fft = torch.fft.rfft(x_norm, dim=1, norm="ortho")
# plt.plot(dct(x, norm='ortho').flatten())
# plt.yscale('log')

# x_norm = (x - x.mean(dim=1)) / torch.sqrt(x.var(dim=1))
freq = torch.fft.rfftfreq(x.shape[1])
print(freq.shape)
# fig, ax = plt.subplots()
# ax.plot(torch.concat([x_fft.real, x_fft.imag[:,1:-1,:]], dim=1).flatten())
# fig.savefig("noisy_0.png")
# plt.close()

mat = [MovingAvgFreq(i, freq=freq).Hw for i in range(2, x.shape[1] + 1)]
mat = torch.concat(mat)
fig, axs = plt.subplots(2)
for i in range(len(freq)):
    axs[0].plot(mat[:,i,:].real)
    axs[1].plot(mat[:,i,:].imag)
    
    
# mat = [MovingAvgTime(i) for i in range(2, x.shape[1] + 1)]
# T = len(mat)
# _, _, alpha_bars = linear_schedule(0, 0, T)
# betas = torch.sqrt(1 - alpha_bars)
# betas

# plt.plot(betas)
# for i, degrade_fn in enumerate(mat):
#     # x_noisy = degrade_fn(x)
#     x_noisy = degrade_fn(x_fft) + torch.randn_like(x_fft) * betas[i]
#     # x_noisy = degrade_fn(x) + torch.randn_like(x) * betas[i]
#     fig, ax = plt.subplots()
#     ax.plot(torch.concat([x_noisy.real, x_noisy.imag[:,1:-1,:]], dim=1).flatten())
#     fig.savefig(f"noisy_{i+1}.png")
#     plt.close()
#     # if i == (T - 1):
#     #     print(norm.fit(x_noisy.numpy().flatten()))

In [ ]:
import torch
import numpy as np
from src.utils.fourier import dft

# Generate a random time-domain signal
N = 1000  # Number of samples
signal = 2 * torch.sin(torch.linspace(0, 10, 1000))

# Compute the DFT of the signal using PyTorch's rfft function with norm='ortho'
x_fft = torch.fft.rfft(signal, norm='ortho')
x_fft = dft(signal.reshape(1,-1,1), real_imag=True).flatten()


# Calculate the mean from the DFT coefficients
mean = x_fft[0].real / torch.sqrt(torch.tensor(N))

# Calculate the variance and standard deviation from the DFT coefficients
energy_freq = (torch.sum(torch.abs(x_fft[1:])**2) * 2 + torch.abs(x_fft[0])**2)  # Account for orthogonal normalization and one-sided representation
adjusted_energy = energy_freq - torch.abs(x_fft[0])**2
variance = adjusted_energy / (N - 1)
std_dev = torch.sqrt(variance)

# Print the results
print("Mean (Time Domain):", torch.mean(signal).item())
print("Mean (Frequency Domain):", mean.item())
print("Standard Deviation (Time Domain):", torch.std(signal).item())
print("Standard Deviation (Frequency Domain):", std_dev.item())


In [ ]:
import torch
# from src.schedules.collection import linear_schedule, cosine_schedule
from src.utils.filters import MovingAvgFreq, MovingAvgTime
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv(
    "/home/user/data/FrequencyDiffusion/dataset/MFRED_clean.csv",
    index_col=0,
    parse_dates=True,
)[["value"]]
df = df[:288]
x = torch.from_numpy(df.values).reshape(1, -1, 1)
freq = torch.fft.rfftfreq(x.shape[1])
x_freq = torch.fft.rfft(x, dim=1, norm='ortho')


maf = MovingAvgFreq(1, freq=freq)
x_freq_flt = maf(x_freq)
x_freq_flt = torch.fft.irfft(x_freq_flt, dim=1, norm='ortho')[:,0::1,:]
x_time_flt = torch.nn.functional.avg_pool1d(x.permute(0,2,1), 1, 1).permute(0,2,1)
print(x_freq_flt)
print(x_time_flt)
plt.plot(x_freq_flt.flatten())
plt.plot(x_time_flt.flatten())
# mat = [MovingAvgTime(i) for i in range(2, x.shape[1] + 1)]

# for i, degrade_fn in enumerate(mat):
#     # x_noisy = degrade_fn(x)
#     x_noisy = degrade_fn(x_fft) + torch.randn_like(x_fft) * betas[i]
#     # x_noisy = degrade_fn(x) + torch.randn_like(x) * betas[i]
#     fig, ax = plt.subplots()
#     ax.plot(torch.concat([x_noisy.real, x_noisy.imag[:,1:-1,:]], dim=1).flatten())
#     fig.savefig(f"noisy_{i+1}.png")
#     plt.close()
#     # if i == (T - 1):
#     #     print(norm.fit(x_noisy.numpy().flatten()))

In [ ]:
from src.utils.filters import MovingAvgTime, MovingAvgFreq
import torch
import matplotlib.pyplot as plt
import pandas as pd
from src.utils.filters import MovingAvgTime, MovingAvgFreq
from src.utils.fourier import complex2sphere, sphere2complex
from scipy.stats import norm

df = pd.read_csv(
    "/home/user/data/FrequencyDiffusion/dataset/MFRED_clean.csv",
    index_col=0,
    parse_dates=True,
)[["value"]]
mode = "linear"
df = df[:288]
kernel_size = 84
# x = torch.randn(12).reshape(1, -1, 1)
x = torch.from_numpy(df.values).reshape(1, -1, 1)
mat = MovingAvgTime(kernel_size, mode=mode)
x_down_real = mat(x)
mat_stride = torch.nn.AvgPool1d(kernel_size, kernel_size)
x_down_real_stride = mat_stride(x.permute(0, 2, 1))

rec = torch.nn.functional.interpolate(
    x_down_real.reshape(1, 1, 1, -1).repeat(10,1,1,1), size=x.shape[1] - kernel_size + 1, mode=mode
)
real = x_down_real_stride.flatten()
plt.plot(rec[:,0,0,::kernel_size])
plt.plot(real.flatten())


In [ ]:
import torch

a = torch.randn(12).reshape(1, -1, 1).float()
avg = torch.nn.AvgPool1d(3,3)
a_avg = avg(a.permute(0,2,1)).permute(0,2,1)

mean = torch.mean(a, dim=1, keepdim=True)
stdev = torch.sqrt(torch.var(a, dim=1, keepdim=True, unbiased=False) + 1e-5)
a_norm = (a - mean) / stdev
a_norm_avg = avg(a_norm.permute(0,2,1)).permute(0,2,1)
torch.allclose(a_norm_avg*stdev + mean, a_avg)

In [ ]:
import torch
a = torch.arange(16).float()

a_fft_norm = torch.fft.rfft(a, norm='ortho')

k1 = 12
k2 = 2

print(a_fft_norm)
b = k1 * a + k2
print(b)
b_fft = a_fft_norm.clone() 
b_fft = b_fft * k1
b_fft[0] += k2 * len(a) ** 0.5 
print(b_fft)
# print(b_fft)
# print(b)
print(torch.fft.irfft(b_fft, norm='ortho'))
# torch.allclose(4*a, torch.fft.irfft(a_fft * 4))

In [4]:
import pandas as pd

df = pd.read_csv(
    "/mnt/ExtraDisk/wcx/research/FrequencyDiffusion/dataset/MFRED_wiztemp.csv", index_col=0, parse_dates=True
)
df = df.interpolate(axis=0)
df = df.reset_index()
df['weekday'] = df['ds'].dt.weekday
df['hour'] = df['ds'].dt.hour
# df
df.to_csv('/mnt/ExtraDisk/wcx/research/FrequencyDiffusion/dataset/MFRED_clean.csv', index=False)
df.to_csv('../dataset/MFRED_clean.csv', index=False)